# PANCAKE PREDICTOR: BNB 5-MINUTE MODEL

A BiLSTM + Attention model for predicting BNB price direction on PancakeSwap's 5-minute prediction rounds.

## CRISP-DM Workflow

### 1. Business Understanding
- **The Game**: Bet Bull (Up) or Bear (Down) for a 5-minute timeframe.
- **The Constraint**: Must place the bet before the round locks.
- **The Edge**: BiLSTMs detect momentum shifts invisible to the naked eye.
- **The House Edge**: PancakeSwap takes ~3%. Model needs >53% accuracy to break even, or play Positive Expected Value.

### 2. Data Understanding & Preparation
- **Market Data**: 1-minute OHLCV candles for BNB/USDT.
- **Contract Data**: Crowd sentiment (Bull Payout vs. Bear Payout).
- **Feature Engineering**: RSI, MACD-style momentum, Log Returns for stationarity.

### 3. The Architecture (DLA Based)
- **Input**: (Batch, 30, Features)
- **BiLSTM Encoder**: Captures momentum (price acceleration/exhaustion).
- **Attention Layer**: Focuses on volume spikes (whale activity).
- **Output**: Sigmoid probability of closing HIGHER than Lock Price.

## Setup & Configuration

In [ ]:
# ==========================================
# PANCAKE PREDICTOR: BNB 5-MINUTE MODEL
# ==========================================

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
SEQ_LENGTH = 30       # Look back at last 30 minutes
PREDICT_AHEAD = 5     # Predict 5 minutes into the future
FEATURES = 5          # [Close, Volume, RSI, Bull_Ratio, Bear_Ratio]

## 1. Data Generator (Simulating BNB Price & Contract Data)

In [ ]:
def generate_market_data(n_minutes=10000):
    """
    Simulates BNB price movement and Prediction Pool Sentiment.
    """
    prices = [300.0]
    volumes = [1000.0]
    bull_ratios = [1.8] # Payout multiplier
    
    for _ in range(n_minutes):
        # Random Walk for Price
        change = np.random.normal(0, 1.5) # Volatility
        new_price = prices[-1] + change
        prices.append(new_price)
        
        # Volume spikes on large moves
        vol = np.random.normal(1000, 200) + (abs(change) * 100)
        volumes.append(vol)
        
        # Pool Sentiment (Crowd usually chases the trend)
        # If price went up, crowd goes Bull (lowering Bull payout)
        sentiment = 1.8 - (change * 0.1) 
        bull_ratios.append(np.clip(sentiment, 1.1, 3.0))
        
    df = pd.DataFrame({
        'Close': prices,
        'Volume': volumes,
        'Bull_Payout': bull_ratios,
        'Bear_Payout': [4.0 - x for x in bull_ratios] # Approximate inverse
    })
    
    # Calculate RSI (Technical Indicator)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss.replace(0, np.finfo(float).eps)
    df['RSI'] = 100 - (100 / (1 + rs))
    df = df.fillna(50)
    
    return df

print(">>> FETCHING ON-CHAIN DATA (Simulated)...")
market_data = generate_market_data()

## 2. Preprocessing (Sequence Creation)

In [ ]:
def create_sequences(df):
    X = []
    y = []
    
    # Normalize Data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    
    # Create Windows
    # We want to use T-30 to T-0 to predict if Price(T+5) > Price(T)
    data_val = df.values
    
    for i in range(SEQ_LENGTH, len(df) - PREDICT_AHEAD):
        # Input: Past 30 mins
        X.append(scaled_data[i-SEQ_LENGTH:i])
        
        # Target: Did price go UP in the next 5 mins?
        current_price = data_val[i][0] # Close is index 0
        future_price = data_val[i + PREDICT_AHEAD][0]
        
        label = 1 if future_price > current_price else 0
        y.append(label)
        
    return np.array(X), np.array(y), scaler

print(">>> PREPARING SEQUENCES...")
X, y, scaler = create_sequences(market_data)
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

print(f"Training Shapes: X={X_train.shape}, y={y_train.shape}")

## 3. The Model (BiLSTM + Attention)

**BiLSTM (Momentum)**: The model reads the last 30 minutes bidirectionally. Strong "Green Candle" patterns with increasing volume bias towards Bull.

**Attention (Volume)**: If a massive sell-off occurred 15 minutes ago, the Attention layer highlights that event, warning the model of overhead resistance.

In [ ]:
def build_pancake_model():
    inputs = Input(shape=(SEQ_LENGTH, FEATURES))
    
    # Layer 1: BiLSTM (Momentum Detector)
    # Reads minute-by-minute price action
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(inputs)
    x = layers.Dropout(0.2)(x)
    
    # Layer 2: Self-Attention (Whale Detector)
    # Focuses on specific minutes with abnormal volume
    attn = layers.MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x = layers.Add()([x, attn])
    x = layers.LayerNormalization()(x)
    
    # Layer 3: Decision
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(32, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid', name="Bull_Probability")(x)
    
    model = models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_pancake_model()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

## 4. Trading Strategy (Inference & Odds Calculation)

### The "Kelly Criterion" (EV Logic)

The `trade_logic` function is crucial. Sometimes the AI is unsure (51% Bull), but if the crowd is heavily betting Bear, the Bull Payout might be 2.5x.

**EV = (0.51 × 2.5) - (0.49 × 1) = 0.78** — a massively profitable bet despite low confidence.

In [ ]:
def trade_logic(current_sequence, bull_payout, bear_payout):
    """
    Decides whether to bet based on Model Confidence AND Pool Odds (EV).
    """
    # 1. Get AI Prediction
    # Reshape for model: (1, 30, 5)
    seq_reshaped = current_sequence.reshape(1, SEQ_LENGTH, FEATURES)
    prob_bull = model.predict(seq_reshaped, verbose=0)[0][0]
    prob_bear = 1.0 - prob_bull
    
    decision = "SKIP"
    ev = 0
    
    # 2. Calculate Expected Value (EV)
    # EV = (Probability * Payout) - (Probability_Loss * Stake)
    
    # Check Bull Case
    ev_bull = (prob_bull * bull_payout) - (prob_bear * 1)
    
    # Check Bear Case
    ev_bear = (prob_bear * bear_payout) - (prob_bull * 1)
    
    # 3. Thresholding (Only bet if we have an edge)
    CONFIDENCE_THRESHOLD = 0.60
    
    if ev_bull > 0.2 and prob_bull > CONFIDENCE_THRESHOLD:
        decision = "BET BULL"
    elif ev_bear > 0.2 and prob_bear > CONFIDENCE_THRESHOLD:
        decision = "BET BEAR"
        
    return decision, prob_bull, ev_bull, ev_bear

# --- SIMULATE A LIVE ROUND ---
print("\n>>> LIVE ROUND PREDICTION ---")
latest_data = X_test[-1] # Pretend this is the current live candle data
current_bull_payout = 1.95 # From Contract
current_bear_payout = 1.95 # From Contract

action, conf, ev_up, ev_down = trade_logic(latest_data, current_bull_payout, current_bear_payout)

print(f"AI Bull Probability: {conf:.2%}")
print(f"EV Bull: {ev_up:.2f} | EV Bear: {ev_down:.2f}")
print(f"STRATEGY CALL: {action}")

## 5. Training Visualization

In [ ]:
# Plot training results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0].axhline(y=0.53, color='r', linestyle='--', label='Break-Even (53%)')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

# Loss
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

plt.tight_layout()
plt.show()

## Real-World Integration Steps

1. **Web3.py**: Fetch `currentEpoch`, `lockPrice`, and `bullAmount`/`bearAmount` from the PancakeSwap Prediction Smart Contract.
2. **Binance API**: Use `ccxt` or `python-binance` to fetch real-time OHLCV data to feed the model.
3. **Latency**: Run close to BSC nodes. The "Lock" happens instantly; your transaction needs to be confirmed 5-10 seconds before the lock.